In [1]:
# Import packages

import TS_Custom
from TS_Custom import *

        
# name of csv file 
backtestingresults_filename = "zlogs_alphaedge_backtestingresults_FULL" + ".csv"
ae_traderesults_filename = "zlogs_alphaedge_backtestingrestuls_ONLY_TRADES" + ".csv"




# Global variables
d = {'a': [], 'b': []}

minutes = 2     
barsBack = 500
#quantity = int(3800/(df['Close'][ind]))
quantity = 5
#barsBack=int(barssince7am(minutes))

def getBacktestingResult(symbol):
    #for symbol in symbol_list :
    try:

        # get price history from tradestation
        
        df = getPriceHistory_TS(symbol,str(barsBack), str(minutes))
        
        #convert current candles to heiken-ashi
        df = getHA(df)
         
        df = Supertrend(df, 10, 1)

        
        df['Position'] = 0
        df['OrderType'] = "" 
        tradetype = ""
        avgprice = ""

        
        for ind in df.index:
            if (ind>3):
                
                if ((df['High'][ind-2] > df['High'][ind-3]) and \
                    (df['Low'][ind-2] < df['Low'][ind-3]) and \
                    (df['High'][ind-1] < df['High'][ind-2]) and \
                    (df['Low'][ind-1] < df['Low'][ind-2])) :

                    dt=datetime.strptime(pd.to_datetime(df['TimeStamp'][ind]).strftime('%m-%d-%Y %H:%M:%S'),\
                                                     '%m-%d-%Y %H:%M:%S')
                    #Buy Condition                                    
                    if (
                        (df['Close'][ind] > df['High'][ind-1]) and \
                        (tradetype=="")):


                        df['Position'][ind] = -quantity
                        df['OrderType'][ind] = "BUYSTOCK"
                        tradetype = "BUYSTOCK"
                        avgprice = df['Close'][ind]
                        
                        continue

                    #Sell Condition
                    if (tradetype=="BUYSTOCK"):

                        openpl = float(pd.to_numeric(df['HA_Close'][ind]) - pd.to_numeric(avgprice))
                        openpl_percent = float(((pd.to_numeric(df['HA_Close'][ind]) - pd.to_numeric(avgprice))/pd.to_numeric(avgprice))*100)

                        if (
                            (df['Close'][ind] < df['Low'][ind-1]) or \
                            (df['Close'][ind] >= df['High'][ind-2]) ):


                            df['Position'][ind] = quantity
                            df['OrderType'][ind] = "SELLSTOCK"
                            tradetype = "" 
                            avgprice = ""     
                            continue

                

                if ((df['High'][ind-2] > df['High'][ind-3]) and \
                    (df['Low'][ind-2] < df['Low'][ind-3]) and \
                    (df['High'][ind-1] < df['High'][ind-2]) and \
                    (df['Low'][ind-1] < df['Low'][ind-2])) :
                    
                    #SellShort Condition
                    if (
                        (df['Close'][ind] < df['High'][ind-1]) and \
                        (tradetype=="")): 

                        df['Position'][ind] = quantity
                        df['OrderType'][ind] = "SELLSHORTSTOCK"
                        tradetype = "SELLSHORTSTOCK"
                        avgprice = df['Close'][ind]

                        continue

                
                    #BuyToCover Condition
                    if (tradetype=="SELLSHORTSTOCK"):

                        openpl = float(pd.to_numeric(avgprice) - pd.to_numeric(df['HA_Close'][ind]))
                        openpl_percent = float(((pd.to_numeric(avgprice) - pd.to_numeric(df['HA_Close'][ind]))/pd.to_numeric(avgprice))*100)

                        #print ('openpl_percent for sell is ',openpl_percent)

                        if (
                            (df['Close'][ind] > df['Low'][ind-1]) or \
                            (df['Close'][ind] <= df['High'][ind-2]) ):

                            df['Position'][ind] = -quantity
                            df['OrderType'][ind] = "BUYTOCOVERSTOCK"
                            tradetype = "" 
                            avgprice = ""
                            continue
    

        df['Value']=df['Position']*df['Close']
        df['Symbol'] = symbol
        df = df[['Symbol','TimeStamp','Close','Open','High','Low','ema9','ema21','ema50','ema200','vwap',
                 'ATR','K_slow','D_slow','ST_Lowerband','ST_Upperband','Position','Value','OrderType']]
        #pd.set_option('display.max_rows', None)
        #print(df)
        
        df2 = df.loc[((df['Position'] == -quantity ) | (df['Position'] == quantity))]
        df2['P/L'] = 0.00
        df2.reset_index(inplace=True)
        for ind in df2.index:
            if (ind>0):
                if ((df2['OrderType'][ind] == "SELLSTOCK") or (df2['OrderType'][ind] == "BUYTOCOVERSTOCK")):
                    df2['P/L'][ind] = df2['Value'][ind] + df2['Value'][ind-1]
                    #ind = ind + 2
            
        profit_loss_symbol = df2['P/L'].sum()        
        print('P/L =',profit_loss_symbol,'for',symbol)
        
        
        d['a'].append(profit_loss_symbol) 
        d['b'].append(df['Close'][0]*quantity)

                
        df.to_csv(backtestingresults_filename, mode = 'a', index=False)
        df2.to_csv(ae_traderesults_filename, mode = 'a', index=False)
        
        time.sleep(1)      
        

    except Exception as e:
        traceback.print_exc()
        print(e)
        


#Backtest Strategy

symbol_list = pd.read_csv('tickers_backtesting.csv', header=None)[0].tolist()
print(symbol_list)



# Time starts from here
start = time.time()

import concurrent.futures

with concurrent.futures.ThreadPoolExecutor(max_workers=len(symbol_list)) as executor:
    #print(threading.currentThread().ident)
    executor.map(getBacktestingResult,symbol_list)

# Total time calculated
print('The program takes ', time.time()-start, 'seconds.')


 
total_profit_loss = sumOfList(d['a'], len(d['a']))
total_cost = sumOfList(d['b'], len(d['b']))

print("---------------------------------------------")
print("Number of Days: ", (barsBack/240))
print("Number of Shares per Stock: ", quantity)
print("Total Profit and Loss: $", round(total_profit_loss,2))
print("Total Cost: $", round(total_cost,2))
print("Total %P/L: ", round(((total_profit_loss/total_cost) * 100),2),'%')

print("---------------------------------------------")
        
print("**************************************************************")
   




['SPY', 'QQQ', 'TSLA', 'AAPL', 'NVDA', 'NFLX']
P/L = 7.250000000000227 for SPY
P/L = -16.149999999999864 for NVDA
P/L = 24.999999999999886 for AAPL
P/L = 19.449999999999818 for TSLA
P/L = 46.65000000000032 for QQQ
P/L = -38.80000000000041 for NFLX
The program takes  2.1540184020996094 seconds.
---------------------------------------------
Number of Days:  2.0833333333333335
Number of Shares per Stock:  5
Total Profit and Loss: $ 43.4
Total Cost: $ 6627.95
Total %P/L:  0.65 %
---------------------------------------------
**************************************************************
